In [1]:
import os
import pandas as pd
from tqdm import tqdm
import random
# 데이터 정보 변수
output_dir = "/data/csafeproject/CSAFE_version5/Document_Forgery"
data_info = 'autoencoder_128d'

train_file = f"mil_{data_info}_train_data.csv"
val_file = f"mil_{data_info}_val_data.csv"
test_file = f"mil_{data_info}_test_data.csv"


train_data = pd.read_csv(os.path.join(output_dir, train_file))
val_data = pd.read_csv(os.path.join(output_dir, val_file))
test_data = pd.read_csv(os.path.join(output_dir, test_file))

print("CSV files loaded successfully!")

CSV files loaded successfully!


In [39]:
train_data = train_data[train_data['repeat'].between(1,18)]
val_data = val_data[val_data['repeat'].between(1,18)]
test_data = test_data[test_data['repeat'].between(1,18)]
train_data

,image_path,label,repeat,text,var_1,var_2,var_3,var_4,var_5,var_6,...,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128
0,/data/csafeproject/another_datas/CSAFE/csafe_v...,300,1,a,-8.106145,0.888668,-6.019303,3.592807,1.695437,-4.117580,...,0.465308,7.022779,1.133619,-7.675983,2.425411,-0.157275,-3.640760,3.996742,5.236913,-6.595477
1,/data/csafeproject/another_datas/CSAFE/csafe_v...,300,1,and,-3.433411,2.057449,-3.499629,-5.862058,-7.490884,-5.840775,...,0.024406,3.072287,1.212291,-1.706382,2.348989,-1.833417,-1.996737,2.446160,-3.350814,-1.163269
2,/data/csafeproject/another_datas/CSAFE/csafe_v...,300,1,away,-2.872294,-2.802950,-3.492892,-5.961239,-2.576890,-5.133574,...,-3.051732,3.710957,2.545245,-5.484296,2.448363,-0.156249,-6.880914,-1.492394,4.296579,-3.090651
3,/data/csafeproject/another_datas/CSAFE/csafe_v...,300,1,barrely,5.116207,-6.018917,-6.738207,-0.260507,0.573614,-5.719716,...,3.902409,6.221342,1.825699,-6.674193,6.281979,-3.987954,-11.019858,0.800846,4.947557,0.876557
4,/data/csafeproject/another_datas/CSAFE/csafe_v...,300,1,been,-5.517964,2.542010,-5.153834,0.940769,0.544770,-6.448665,...,-4.734687,6.505259,4.007088,-10.108008,1.759517,-0.041626,2.745936,3.110706,3.984141,-2.523834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135290,/data/csafeproject/another_datas/CSAFE/csafe_v...,414,18,turinand,-9.077224,9.611026,4.462167,9.350149,-1.046566,-2.437427,...,-3.569644,-4.484183,-4.365593,-3.873962,-6.784763,9.344830,3.135248,2.898360,0.088359,1.137584
135291,/data/csafeproject/another_datas/CSAFE/csafe_v...,414,18,we,-10.477049,3.380688,3.894536,0.139162,-3.578593,-3.281197,...,-5.716061,4.512581,1.445635,4.231771,-7.022221,-1.353917,-0.106289,-3.835144,-6.765935,-11.252061
135292,/data/csafeproject/another_datas/CSAFE/csafe_v...,414,18,week,-9.336310,3.397638,-0.422345,3.049784,0.094342,-6.349130,...,-9.009175,1.603983,0.505769,-6.391850,-1.026117,7.193019,-1.845734,3.924395,0.868310,-5.851886
135293,/data/csafeproject/another_datas/CSAFE/csafe_v...,414,18,will,-13.454964,11.394653,0.411629,4.538775,-1.715419,-1.312487,...,-3.197354,-6.037574,-4.513397,-0.329747,-11.318637,3.887090,-5.897219,3.352936,-2.215656,3.709592


In [21]:
pd.set_option('display.max_rows', 80)

In [28]:
def generate_bags(data, replacement_counts=[1, 2, 3, 4, 5]):
    features_bags = []

    # 그룹화: 'repeat'와 'label'을 기준으로 그룹화
    grouped = data.groupby(['repeat', 'label'])

    # False Bags 생성
    for (repeat, label), group in grouped:
        bag_id = f"False_{label}_{repeat}"
        metadata = {
            'label': label,
            'repeat': repeat,
            'original_indices': group.index.tolist()  # Bag 내 원본 데이터 인덱스 저장
        }
        features = group[[f'var_{i}' for i in range(1, 129)]].values  # feature columns
        bag = {
            'bag_id': bag_id,
            'label': False,
            'metadata': metadata,
            'features': features.tolist()
        }
        features_bags.append(bag)

    train_false_bags = pd.DataFrame(features_bags)

    # True Bags 생성
    true_bags = []

    for (repeat, label), group in tqdm(grouped, desc=f"Processing Groups", total=len(grouped)):
        for replacement_count in replacement_counts:
            bag_id = f"True_{label}_{repeat}_{replacement_count}"
            false_bag = group.copy()

            # false_bag의 인덱스를 랜덤하게 섞음
            shuffled_indices = list(false_bag.index)
            random.shuffle(shuffled_indices)

            modified_indices = []  # 교체한 데이터 인덱스 저장
            changes = []  # 원본-교체된 인덱스 매핑 저장
            updated_indices = false_bag.index.tolist()  # 교체 후 인덱스를 업데이트하기 위한 리스트

            for idx in shuffled_indices:
                row = false_bag.loc[idx]

                # 동일한 text를 가진 다른 label의 데이터 찾기
                matching_text = data[(data['text'] == row['text']) & (data['label'] != row['label'])]

                if not matching_text.empty:
                    # 동일한 text를 가진 데이터 중 랜덤 선택
                    replacement = matching_text.sample(n=1).iloc[0]
                else:
                    # 동일한 text가 없는 경우, 다른 label의 데이터를 랜덤 선택
                    replacement = data[data['label'] != row['label']].sample(n=1).iloc[0]

                # Bag 내에서 몇 번째 벡터인지 계산
                position_in_bag = list(false_bag.index).index(idx)

                # **feature 교체: 해당 row의 feature 값을 replacement로 업데이트**
                for col in [f'var_{i}' for i in range(1, 129)]:
                    false_bag.at[idx, col] = replacement[col]

                # Bag 내에서 해당 인덱스 정보를 업데이트
                updated_indices[position_in_bag] = replacement.name

                # 저장: 원본 인덱스, 교체된 데이터의 인덱스, Bag 내 위치
                modified_indices.append(idx)
                changes.append({
                    "original_index": idx,
                    "replaced_index": replacement.name,
                    "position_in_bag": position_in_bag,
                    "new_label": replacement['label']  # 교체된 데이터의 라벨 추가
                })

                # 교체한 데이터 개수가 목표 개수에 도달하면 종료
                if len(modified_indices) >= replacement_count:
                    break

            # True bag 생성 및 추가
            if modified_indices:
                metadata = {
                    "label": label,
                    "repeat": repeat,
                    "text_changes": len(modified_indices),  # 교체한 데이터 개수 추가
                    "index_mapping": changes,               # 원본-교체된 인덱스 매핑 추가
                    "all_indices": updated_indices          # 교체 결과를 반영한 인덱스 리스트
                }
                features = false_bag[[f'var_{i}' for i in range(1, 129)]].values
                bag = {
                    'bag_id': bag_id,
                    'label': True,
                    'metadata': metadata,
                    'features': features.tolist()
                }
                true_bags.append(bag)

    train_true_bags = pd.DataFrame(true_bags)

    # False Bags와 True Bags 합치기
    combined_bags = pd.concat([train_false_bags, train_true_bags], ignore_index=True)
    return combined_bags

In [29]:
train_bags = generate_bags(train_data, replacement_counts=[1])
val_bags = generate_bags(val_data, replacement_counts=[1])
test_bags = generate_bags(test_data, replacement_counts=[1])

Processing Groups: 100%|██████████| 810/810 [00:10<00:00, 77.87it/s]


In [34]:
train_bags.iloc[2,]['metadata']

{'label': 302,
 'repeat': 1,
 'original_indices': [2436,
  2437,
  2438,
  2439,
  2440,
  2441,
  2442,
  2443,
  2444,
  2445,
  2446,
  2447,
  2448,
  2449,
  2450,
  2451,
  2452,
  2453,
  2454,
  2455,
  2456,
  2457,
  2458,
  2459,
  2460,
  2461,
  2462,
  2463,
  2464,
  2465,
  2466,
  2467,
  2468,
  2469,
  2470,
  2471,
  2472,
  2473,
  2474,
  2475,
  2476,
  2477,
  2478,
  2479,
  2480,
  2481,
  2482,
  2483,
  2484,
  2485,
  2486,
  2487,
  2488,
  2489,
  2490,
  2491,
  2492]}

In [35]:
pd.DataFrame(train_bags.iloc[2,]['features'])

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-4.879781,-5.877129,3.239186,2.827141,-1.776151,6.730612,5.634574,4.635252,-0.300920,-1.045150,...,0.632417,-2.314332,-2.594263,9.058727,-1.998481,-3.012756,-3.143657,-2.750071,-4.658449,-10.717886
1,4.031734,-0.908371,-2.092624,-0.468493,-3.906314,5.625807,4.557531,3.759680,-4.073231,-8.147000,...,7.562937,-4.094532,-1.677008,7.337509,-6.813694,-12.140633,-9.412933,-8.554355,-2.036519,6.854702
2,1.178046,-8.776061,1.141188,3.427043,7.192419,-2.288738,-0.648313,3.990868,4.558999,-4.258408,...,6.715294,10.665365,7.340296,1.313976,3.740271,-6.302510,-6.482983,-3.161418,4.076174,5.280829
3,3.155813,-6.530759,3.147895,4.010315,-0.952684,4.675696,1.966542,3.646191,1.882891,-2.647195,...,4.207099,-0.572294,0.222809,8.688829,-0.419955,-6.702423,0.411197,-5.108657,1.511051,-5.523768
4,-0.580876,-6.001217,-3.948028,-4.468914,-3.378079,-1.642725,7.259910,-1.754241,5.970041,0.032408,...,2.546555,6.487779,3.765585,-0.774118,2.090280,-7.994141,-4.324531,-6.169010,1.316734,0.259665
5,0.545402,-11.265877,3.200385,0.047087,2.414403,1.709960,1.717397,-1.337367,2.010265,4.908518,...,-1.636696,-1.207001,-0.278952,-3.177184,0.944514,-2.744144,-0.915138,-6.135797,5.542316,1.333474
6,-3.769214,-1.465799,3.626070,0.113021,8.958162,8.866839,-2.525350,2.311068,7.215519,2.633702,...,-4.030345,-2.792185,-0.114212,1.119519,-1.609595,-6.931135,-2.024316,-4.854936,6.498237,3.416417
7,-3.291720,-9.707803,1.447983,4.184610,4.661347,6.483086,2.382554,-0.012515,8.681444,1.640035,...,-2.363210,-1.030811,-5.017759,6.238159,-2.070498,-7.633980,2.277034,-6.657199,5.139216,-8.392159
8,-6.735332,-3.184678,4.285990,1.858221,4.381495,8.764254,-2.397551,6.299208,9.862565,0.437716,...,-5.772313,-2.154726,-1.078592,11.012705,-6.854709,-7.805445,-1.978038,-6.889675,-4.474951,-7.526667
9,-4.567139,3.489328,2.675353,0.785014,0.206819,7.369148,3.392837,2.072967,7.974890,-3.336289,...,-1.439011,0.967860,0.784321,5.119738,-3.946098,-9.687166,-2.641730,-6.178140,-2.844504,-2.653841


In [23]:
train_bags.to_pickle("/data/csafeproject/CSAFE_version5/Document_Forgery/train_bags_autoencoder_128d.pkl")
val_bags.to_pickle("/data/csafeproject/CSAFE_version5/Document_Forgery/val_bags_autoencoder_128d.pkl")
test_bags.to_pickle("/data/csafeproject/CSAFE_version5/Document_Forgery/test_bags_autoencoder_128d.pkl")